<a href="https://colab.research.google.com/github/15483114/DockerQuest/blob/main/2015_Flight_Delays_and_Cancellations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2015 Flight Delays and Cancellations**

Neste notebook será analisado o dataset de atrasos e cancelamentos de voos em 2015.

O objetivo é descobrir se é possível obter um bom modelo de classificação para prever se um vôo irá atrasar ou não dadas as variaáveis coletadas para a criação do dataset.

# Importando Dataset

In [ ]:
import pandas as pd

# airlines = pd.read_csv('/content/airlines.csv')
# airports = pd.read_csv('/content/airports.csv')
flights = pd.read_csv('/content/flights.csv', error_bad_lines=False)

b'Skipping line 1848378: expected 31 fields, saw 42\n'
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Explorando os datasets

In [ ]:
flights['DIVERTED'].describe()

In [ ]:
flights[["DIVERTED"]].value_counts(sort = False)

In [ ]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1363989 entries, 0 to 1372159
Data columns (total 25 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   MONTH                1363989 non-null  int64  
 1   DAY                  1363989 non-null  int64  
 2   DAY_OF_WEEK          1363989 non-null  int64  
 3   AIRLINE              1363989 non-null  int64  
 4   FLIGHT_NUMBER        1363989 non-null  int64  
 5   TAIL_NUMBER          1363989 non-null  int64  
 6   ORIGIN_AIRPORT       1363989 non-null  int64  
 7   DESTINATION_AIRPORT  1363989 non-null  int64  
 8   SCHEDULED_DEPARTURE  1363989 non-null  int64  
 9   DEPARTURE_TIME       1363989 non-null  float64
 10  DEPARTURE_DELAY      1363989 non-null  float64
 11  TAXI_OUT             1363989 non-null  float64
 12  WHEELS_OFF           1363989 non-null  float64
 13  SCHEDULED_TIME       1363989 non-null  float64
 14  ELAPSED_TIME         1363989 non-null  float64
 15

In [ ]:
from matplotlib import pyplot as plt
departures = flights['DEPARTURE_TIME']
fig, ax = plt.subplots(figsize =(10, 10))
ax.hist(departures)
 
# Show plot
plt.show()

In [ ]:
pd.plotting.scatter_matrix(flights)

# Lidando com a falta de dados

In [1]:
# Porcentagem de dados faltantes
flights.isnull().mean().round(4)*100

NameError: ignored

In [ ]:
#tirando colunas com mais de 80% das dados faltando
flights = flights.drop(['YEAR'], axis=1) # tirando year também porque é uma constante basicamente
flights = flights.drop(['CANCELLATION_REASON'], axis=1)
flights = flights.drop(['AIR_SYSTEM_DELAY'], axis=1)
flights = flights.drop(['SECURITY_DELAY'], axis=1)
flights = flights.drop(['AIRLINE_DELAY'], axis=1)
flights = flights.drop(['LATE_AIRCRAFT_DELAY'], axis=1)
flights = flights.drop(['WEATHER_DELAY'], axis=1)

In [ ]:
flights.columns

In [ ]:
# Procurando outliers nas colunas com alguma porcentagem de valores faltantes

import plotly.express as px
# df = px.flights.tips()
missing_values = ["DEPARTURE_TIME", "DEPARTURE_DELAY","TAXI_OUT", "WHEELS_OFF", "ELAPSED_TIME", "AIR_TIME", "WHEELS_ON", "TAXI_IN", "ARRIVAL_TIME", "ARRIVAL_DELAY" ]

for yvar in missing_values:
  fig = px.box(flights, y= yvar)
  fig.show()


A estratégia escolhida para lidar com a falta de dados é a substituição dos valores nulos pela média ou mediana dependendo a quantidade de outliers encontrados nos dados.

As features sem outliers receberão a média, estas são: departure_time, wheels_of, wheels_on, arrival_time

As features com outliers receberão a mediana, estas são: departure_delay, taxi_out, elapsed_time, air_time, taxi_in, arrival_delay

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# imputer.fit(flights['DEPARTURE_TIME', 'WHEELS_OF', 'WHEELS_ON', 'ARRIVAL_TIME'])

flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].fillna(flights['DEPARTURE_TIME'].mean())
flights['WHEELS_OFF'] = flights['WHEELS_OFF'].fillna(flights['WHEELS_OFF'].mean())
flights['WHEELS_ON'] = flights['WHEELS_ON'].fillna(flights['WHEELS_ON'].mean())
flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].fillna(flights['ARRIVAL_TIME'].mean())

flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].fillna(flights['DEPARTURE_TIME'].median())
flights['TAXI_OUT'] = flights['TAXI_OUT'].fillna(flights['TAXI_OUT'].median())
flights['ELAPSED_TIME'] = flights['ELAPSED_TIME'].fillna(flights['ELAPSED_TIME'].median())
flights['AIR_TIME'] = flights['AIR_TIME'].fillna(flights['AIR_TIME'].median())
flights['TAXI_IN'] = flights['TAXI_IN'].fillna(flights['TAXI_IN'].median())
flights['ARRIVAL_DELAY'] = flights['ARRIVAL_DELAY'].fillna(flights['ARRIVAL_DELAY'].median())
flights['DEPARTURE_DELAY'] = flights['DEPARTURE_DELAY'].fillna(flights['DEPARTURE_DELAY'].median())

In [ ]:
#medias
list(flights.columns).index('DEPARTURE_TIME',)
list(flights.columns).index('WHEELS_OFF',)
list(flights.columns).index('WHEELS_ON',)
list(flights.columns).index('ARRIVAL_TIME',)
# # mediana
list(flights.columns).index('DEPARTURE_DELAY',)
list(flights.columns).index('TAXI_OUT',)
list(flights.columns).index('ELAPSED_TIME',)
list(flights.columns).index('AIR_TIME',)
list(flights.columns).index('TAXI_IN',)
list(flights.columns).index('ARRIVAL_DELAY',)

In [ ]:
#verificando resultados das substituições
flights.isnull().mean().round(4)*100

In [ ]:
# dropping all nan from tail number
flights.dropna(inplace=True)

In [ ]:
# Vendo a matriz de correlação após tratamento

import plotly.express as px
fig = px.imshow(flights.corr())
fig.show()

Sabendo que a variável target é ARRIVAL_DELAY estas são as vari

TAXI_OUT : 0,227 

DEPARTURE_DELAY: 0,937

DEPARTURE_TIME: 0,159

WHEELS_OFF: 0,155

SCHEDULED_DEPARTURE: 0,098

Removendo variaveis numéricas que não possuem grande correlação com a variável target

# Encoding categorical data

Dados categóricos restantes:

- MONTH                   
- DAY                     
- DAY_OF_WEEK             
- AIRLINE                 
- FLIGHT_NUMBER           
- TAIL_NUMBER             
- ORIGIN_AIRPORT          
- DESTINATION_AIRPORT 

In [ ]:
list(flights.columns).index('MONTH',)
list(flights.columns).index('DAY',)
list(flights.columns).index('DAY_OF_WEEK',)
# list(flights.columns).index('WHEELS_ON',)
# list(flights.columns).index('DESTINATION_AIRPORT',)

3

Criando dummies para month, day e day_of_week por terem poucas categorias

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
flights = np.array(ct.fit_transform(flights))
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [6,7])], remainder='passthrough')
# flights = np.array(ct.fit_transform(flights))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
flights['MONTH'] = le.fit_transform(flights['MONTH'])
flights['DAY'] = le.fit_transform(flights['DAY'])
flights['DAY_OF_WEEK'] = le.fit_transform(flights['DAY_OF_WEEK'])
flights['AIRLINE'] = le.fit_transform(flights['AIRLINE'])
flights['FLIGHT_NUMBER'] = le.fit_transform(flights['FLIGHT_NUMBER'])
flights['TAIL_NUMBER'] = le.fit_transform(flights['TAIL_NUMBER'])
flights['ORIGIN_AIRPORT']=flights['ORIGIN_AIRPORT'].astype(str)
flights['ORIGIN_AIRPORT'] = le.fit_transform(flights['ORIGIN_AIRPORT'])
flights['DESTINATION_AIRPORT']=flights['DESTINATION_AIRPORT'].astype(str)

In [ ]:
flights['DAY_OF_WEEK']

Tirando variáveis categóricas com categorias demais

- Não consegui usar o one hot encoder para criar dummie variables



In [ ]:
flights = flights.drop(['AIRLINE'], axis=1)
flights = flights.drop(['FLIGHT_NUMBER'], axis=1)
flights = flights.drop(['TAIL_NUMBER'], axis=1)
flights = flights.drop(['ORIGIN_AIRPORT'], axis=1)
flights = flights.drop(['DESTINATION_AIRPORT'], axis=1)


In [ ]:
flights.columns

Index(['MONTH', 'DAY', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME',
       'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME',
       'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN',
       'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED',
       'CANCELLED'],
      dtype='object')

Criando a coluna que indica se o vôo atrasou ou não (0 se sim e 1 caso não)

In [ ]:
array=[]
for x in flights['ARRIVAL_DELAY']:
  variable = 0 if x>0 else 1
  array.append(variable)

flights['ARRIVAL_DELAY'] = array

# Dividindo o dataset em treino e teste

In [ ]:
from sklearn.model_selection import train_test_split
var_indexes =[3,4,5,6,7,8,9,10,11,12,13,14,15,17,18]
X = flights.iloc[:, var_indexes].values
y = flights['ARRIVAL_DELAY'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [ ]:
flights.columns

# Feature Scalling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_test

array([[ 1.2719537 ,  1.23416608, -0.17185834, ...,  1.1921276 ,
        -0.05101285, -0.12510682],
       [-1.05380666, -0.81061864,  1.71231913, ..., -0.71550618,
        -0.05101285, -0.12510682],
       [ 0.36645767,  0.37596087,  0.15114351, ...,  0.63161767,
        -0.05101285, -0.12510682],
       ...,
       [ 1.86114633,  1.79018636, -0.41410973, ...,  1.61203503,
        -0.05101285, -0.12510682],
       [ 1.08589287,  1.03472402, -0.41410973, ...,  1.23012827,
        -0.05101285, -0.12510682],
       [ 0.40366984,  0.3739463 , -0.36027609, ...,  0.51381559,
        -0.05101285, -0.12510682]])

# Aplicando e selecionando modelos de regressão

In [ ]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 1]
 ...
 [0 1]
 [0 0]
 [0 1]]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[175003      0]
 [280306   9858]]


0.39740781267802744

In [ ]:
# Random forest

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 5, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=5, random_state=0)

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [1 1]
 [1 1]
 ...
 [1 1]
 [1 0]
 [1 1]]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[160522  14481]
 [  9871 280293]]


0.9476489088864859

In [ ]:
# SVM

# o SVM está demorando muito então pensei em diminuir a quantidade de teste para ele terminar de rodar
X_SVM_train, X_SVM_test, y_SVM_train, y_SVM_test = train_test_split(X, y, test_size = 0.5, random_state = 1)

from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_SVM_train, y_SVM_train)


In [ ]:
y_SVM_pred = classifier.predict(X_SVM_test)
print(np.concatenate((y_SVM_pred.reshape(len(y_SVM_pred),1), y_test.reshape(len(y_SVM_test),1)),1))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_SVM_test, y_pred)
print(cm)
accuracy_score(y_SVM_test, y_pred)

# Avaliando o Modelo


Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

# **Considerações finais**



Foram testados dois modelos de classificação random forest e naive bayes. 
O reultado do Naives bayes foi um pouco baixo, com 39,7%, porém não apresentou falsos negativos, o que é importante caso a classificação errônea seja crítica para o planejamentos dos vôos.

O resultado so random forest foi considerado bom, com acurácia de 94,7%, com alguns erros falsos positivos, porém pode ser interessante caso a classificação de falsos positivos não seja crítica para o planejamento dos vôos.